In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!cp -r /kaggle/input/yolov5sjzz/models/ .
!cp -r /kaggle/input/yolov5sjzz/utils/ .
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/arialttf-2/Arial.ttf /root/.config/Ultralytics/
!ls



In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import torch
import os
import time
import torchvision
import cv2
from PIL import Image

from utils.general import non_max_suppression,scale_coords
from utils.augmentations import letterbox
from models.experimental import attempt_load


for dirname, _, filenames in os.walk('/kaggle/input/yolov5mmodel/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

@torch.no_grad()
def predict(model,
        device,
        im0s,  # file/dir/URL/glob, 0 for webcam
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        half=False,  # use FP16 half-precision inference
        ):
    pt = True
    stride = int(model.stride.max())  # model stride
#     print("ssssss",stride,pt)

    # Dataloader
    
    # dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
    bs = 1  # batch_size
    
    # torch.save(model, 'yolov5s.pth')

    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, *imgsz).to(device).type_as(next(model.parameters())))  # run once

    img = letterbox(im0s, imgsz, stride, pt)[0]
    img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img = np.ascontiguousarray(img)



    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img = img / 255.0  # 0 - 255 to 0.0 - 1.0
    if len(img.shape) == 3:
        img = img[None]  # expand for batch dim

    # Inference

    visualize = False
    pred = model(img, augment=augment, visualize=visualize)[0]

    # NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
    if len(pred)>0:
        pred[0][:, :4] = scale_coords(img.shape[2:], pred[0][:, :4], im0s.shape).round()
        return im0s,pred[0].cpu().numpy()
    else:
        return im0s,[]

    
def format_prediction(bboxes):
    annot = ''
    if bboxes.shape[0]>0:
        for idx in range(bboxes.shape[0]):
            xmin, ymin, xmax, ymax = bboxes[idx,:4]
            conf             = bboxes[idx,4]
            w = int(xmax-xmin)
            h = int(ymax-ymin)
            xmin = int(xmin)
            ymin = int(ymin)
            
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

* 1. 

In [ ]:

CKPT_PATH = '/kaggle/input/yolov5mmodel/yolov5m.pt'
imgsz=[768,768]  # inference size (pixels)
conf_thres=0.15  # confidence threshold
iou_thres=0.45  # NMS IOU threshold
max_det=1000  # maximum detections per image

device = torch.device('cuda:0')
# model = torch.load(CKPT_PATH,map_location=device)
source = '/kaggle/input/yolov5sjzz/test_pic/100.jpg'
image = cv2.imread(source)
images,dets = predict(model,  # model.pt path(s)
                  device,
                 image,  # file/dir/URL/glob, 0 for webcam
                classes=None,  # filter by class: --class 0, or --class 0 2 3
                agnostic_nms=False,  # class-agnostic NMS
                augment=False,  # augmented inference
                visualize=False,  # visualize features
                half=False)  # use FP16 half-precision inference)
    
#     if dets.shape[0]>0:
#         for i in range(dets.shape[0]):
#             cv2.rectangle(images,(int(dets[i,0]),int(dets[i,1])),(int(dets[i,2]),int(dets[i,3])),(255,0,0),2)
#         cv2.imwrite("shjdcugs.jpg",images)


print(dets.shape,dets)
print(format_prediction(dets))

In [ ]:
import greatbarrierreef
import tqdm
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission



In [ ]:

import cv2
from PIL import Image

for (pixel_array, sample_prediction_df) in iter_test:
    img = cv2.cvtColor(pixel_array,cv2.COLOR_RGB2BGR)
    images,dets = predict(model,  # model.pt path(s)
                  device,
                 img,  # file/dir/URL/glob, 0 for webcam
                classes=None,  # filter by class: --class 0, or --class 0 2 3
                agnostic_nms=False,  # class-agnostic NMS
                augment=False,  # augmented inference
                visualize=False,  # visualize features
                half=False)  # use FP16 half-precision inference)
    res_str = format_prediction(dets)
    sample_prediction_df['annotations'] = res_str  # make your predictions here
    print(sample_prediction_df)
    env.predict(sample_prediction_df)   # register your predictions
#     display(Image.fromarray(pixel_array))



# for (pixel_array, sample_prediction_df) in iter_test:
#     sample_prediction_df['annotations'] = '0.5 0 0 100 100'  # make your predictions here
#     env.predict(sample_prediction_df)   # register your predictions
# #     display(Image.fromarray(pixel_array))
#     img = cv2.cvtColor(pixel_array,cv2.COLOR_RGB2BGR)
#     print(img)


    
# sub_df = pd.read_csv('submission.csv')
# sub_df.head()
    

In [ ]:
!ls -lh
!cat submission.csv
# !rm submission.csv